# Dependency Inversion Principle

Um problema muito comum, é termos métodos de baixo nível instanciados em classes de alto nível. Isso prejudica o processo de manutenção, visto que alterações na classe de baixo nível irão quebrar as classes de alto nível que consomem sua estrutura.

In [7]:
from abc import abstractmethod
from enum import Enum


class Relationship(Enum):
    PARENT = 0
    CHILD = 1
    SIBLING = 2


class Person:
    def __init__(self, name):
        self.name = name



class Relationships:  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))

In [8]:
class Research:
#     dependency on a low-level module directly
#     bad because strongly dependent on e.g. storage type

    def __init__(self, relationships):
        # high-level: find all of john's children
        relations = relationships.relations
        for r in relations:
            if r[0].name == 'John' and r[1] == Relationship.PARENT:
                print(f'John has a child called {r[2].name}.')

Veja que a classe Research realiza uma busca utilizando o conceito de lista de Relationships. Caso ocorra uma alteração na estrutura de Relationships, a classe Research ficara comprometida.

In [11]:
parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris.
John has a child called Matt.
John has a child called Chris.
John has a child called Matt.
John has a child called Chris.
John has a child called Matt.


O correto é abstrair o método de pesquisa dentro da classe de baixo nível Relationships. Assim, se ocorrer uma mudança em sua estrutura, como utilização de um dicionário por exemplo, para classes que dependem dessa função, o processo será transparente

In [12]:
class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append((parent, Relationship.PARENT, child))
        self.relations.append((child, Relationship.PARENT, parent))
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r[0].name == name and r[1] == Relationship.PARENT:
                yield r[2].name

class Research:
    
    def __init__(self, browser):
        for p in browser.find_all_children_of("John"):
            print(f'John has a child called {p}')


In [13]:
parent = Person('John')
child1 = Person('Chris')
child2 = Person('Matt')

# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris
John has a child called Matt


Nesse modelo, veja que se alterarmos de lista para dicionário no lugar da tupla, a classe Research não é afetada

In [14]:
class RelationshipBrowser:
    @abstractmethod
    def find_all_children_of(self, name): pass


class Relationships(RelationshipBrowser):  # low-level
    relations = []

    def add_parent_and_child(self, parent, child):
        self.relations.append({'person': parent, 'rel':Relationship.PARENT,
                               'person_l': child})
        self.relations.append({'person': child, 'rel': Relationship.PARENT,
                               'person_l': parent})
            
    def find_all_children_of(self, name):
        for r in self.relations:
            if r['person'].name == name and r['rel'] == Relationship.PARENT:
                yield r['person_l'].name

Veja que a busca continua funcionando da mesma forma utilizando a nova definição de Relationships

In [15]:
# low-level module
relationships = Relationships()
relationships.add_parent_and_child(parent, child1)
relationships.add_parent_and_child(parent, child2)

Research(relationships)

John has a child called Chris
John has a child called Matt
